In [1]:
# Analysis of Retirement with Mutual Funds
# Analysis period from 1999 Jan to 2020 May

#Importing libraries
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [2]:
# install xlrd to read spreadsheets
!pip install xlrd
import xlrd
print('xlrd installed!')

xlrd installed!


In [3]:
# reading excel file into data frame
df_data = pd.read_excel('nav_history.xlsx',
                       sheet_name='Sheet1')

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [4]:
# checking data frame
df_data.head()

,Date,Liquid Fund – NAV,Equity Fund – NAV
0,"Jan 01, 1999",11.41,9.29
1,"Feb 01, 1999",11.52,10.23
2,"Mar 01, 1999",11.62,11.64
3,"Apr 05, 1999",11.74,12.62
4,"May 03, 1999",11.84,11.14


In [5]:
# view df details
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               257 non-null    object 
 1   Liquid Fund – NAV  257 non-null    float64
 2   Equity Fund – NAV  257 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.1+ KB


In [6]:
# Initial monthly income at 1999 Jan 1 is set to ₹12,000. Reason: at 7% inflation, ₹12,000 ~ ₹50,000 as on 2020
# Initial investment is set so that total withdrawal per year is 4% of capital. in this case ₹36 Lakhs
# Asset allocation is set to 60% Equity and 40% Debt

init_inv=3600000
init_eq_per=0.6
init_eq=init_inv*init_eq_per
init_dt=init_inv-init_eq

monthly_income=[] # creating a list for storing monthy income for every year
year_num=0 # to track year in progress
monthly_income.append(12000)

In [19]:

# adding 6 new empty columns
df_data['Liquid fund – Units']=""
df_data['Equity fund – Units']=""
df_data['Redemption']=""
df_data['Liquid fund – Value']=""
df_data['Equity fund – Value']=""
df_data['Equity percent']=""

#initializing first row
df_data.loc[0,'Liquid fund – Units']=(init_dt-monthly_income[year_num])/df_data.loc[0,'Liquid Fund – NAV']
df_data.loc[0,'Equity fund – Units']=init_eq/df_data.loc[0,'Equity Fund – NAV']
df_data.loc[0,'Redemption']=monthly_income[year_num]
df_data.loc[0,'Liquid fund – Value']=df_data.loc[0,'Liquid fund – Units']*df_data.loc[0,'Liquid Fund – NAV']
df_data.loc[0,'Equity fund – Value']=df_data.loc[0,'Equity fund – Units']*df_data.loc[0,'Equity Fund – NAV']
df_data.loc[0,'Equity percent']=(100*df_data.loc[0,'Equity fund – Value'])/(df_data.loc[0,'Equity fund – Value']+df_data.loc[0,'Liquid fund – Value'])

# number of rows
var_rows=len(df_data.index)

for i in range(1,var_rows):
    if(i%12==11): # when 1 year is completed, asset rebalance is triggered
        year_num+=1 # incrementing year number
        # calculating asset allocation before redemption and rebalancing
        df_data.loc[i,'Liquid fund – Units']=df_data.loc[i-1,'Liquid fund – Units']
        df_data.loc[i,'Equity fund – Units']=df_data.loc[i-1,'Equity fund – Units']
        df_data.loc[i,'Liquid fund – Value']=df_data.loc[i,'Liquid fund – Units']*df_data.loc[i,'Liquid Fund – NAV']
        df_data.loc[i,'Equity fund – Value']=df_data.loc[i,'Equity fund – Units']*df_data.loc[i,'Equity Fund – NAV']
        total_value=df_data.loc[i,'Liquid fund – Value']+df_data.loc[i,'Equity fund – Value']
        df_data.loc[i,'Equity percent']=(100*df_data.loc[i,'Equity fund – Value'])/total_value
        # if equity exceeds 60%
        if(df_data.loc[i,'Equity percent']>60):
            # equity is above 60%, moving excess equity asset into liquid fund to bring back asset allocation to 60:40
            excess_equity=df_data.loc[i,'Equity fund – Value']-(0.6*total_value)
            df_data.loc[i,'Liquid fund – Units']=df_data.loc[i,'Liquid fund – Units']+(excess_equity/df_data.loc[i,'Liquid Fund – NAV'])
            df_data.loc[i,'Equity fund – Units']=df_data.loc[i,'Equity fund – Units']-(excess_equity/df_data.loc[i,'Equity Fund – NAV'])
            df_data.loc[i,'Liquid fund – Value']=df_data.loc[i,'Liquid fund – Units']*df_data.loc[i,'Liquid Fund – NAV']
            df_data.loc[i,'Equity fund – Value']=df_data.loc[i,'Equity fund – Units']*df_data.loc[i,'Equity Fund – NAV']
        # Finding new monthly withdrawal
        monthly_income.append(df_data.loc[i,'Liquid fund – Value']/120) # 10% of liquid fund value is redeemed an year
        # recording updated liquid fund units, value and equity percentage after 1 month withdrawal
        df_data.loc[i,'Liquid fund – Units']=df_data.loc[i,'Liquid fund – Units']-(monthly_income[year_num]/df_data.loc[i,'Liquid Fund – NAV'])
        df_data.loc[i,'Liquid fund – Value']=df_data.loc[i,'Liquid fund – Units']*df_data.loc[i,'Liquid Fund – NAV']
        df_data.loc[i,'Equity percent']=(100*df_data.loc[i,'Equity fund – Value'])/(df_data.loc[i,'Equity fund – Value']+df_data.loc[i,'Liquid fund – Value'])
        
    else:
        #code
        df_data.loc[i,'Liquid fund – Units']=df_data.loc[i-1,'Liquid fund – Units']-(monthly_income[year_num]/df_data.loc[i,'Liquid Fund – NAV'])
        df_data.loc[i,'Equity fund – Units']=df_data.loc[i-1,'Equity fund – Units']
        df_data.loc[i,'Redemption']=monthly_income[year_num]
        df_data.loc[i,'Liquid fund – Value']=df_data.loc[i,'Liquid fund – Units']*df_data.loc[i,'Liquid Fund – NAV']
        df_data.loc[i,'Equity fund – Value']=df_data.loc[i,'Equity fund – Units']*df_data.loc[i,'Equity Fund – NAV']
        df_data.loc[i,'Equity percent']=(100*df_data.loc[i,'Equity fund – Value'])/(df_data.loc[i,'Equity fund – Value']+df_data.loc[i,'Liquid fund – Value'])
    #end else
#end for

In [23]:
monthly_income

[12000,
 21150.871765997643,
 21299.190333506813,
 20790.229786530803,
 20936.01938490954,
 20581.327342160406,
 20701.80637913786,
 34983.68974785774,
 40241.621850978285,
 55604.83557410367,
 70169.82673312609,
 86384.77358189163,
 85110.82647011589,
 89204.49283201185,
 104853.14362824633,
 103116.96583464602,
 102216.22687513399,
 101381.36253652693,
 140472.8616968913,
 137764.85964435062,
 138428.1697854247,
 160584.71964032488,
 157497.1284336916,
 162890.93252547825]